In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

C:\Users\virendra\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [28]:
#initializing the CNn
import pickle

In [3]:
classifier=Sequential()

In [4]:
#convolution=(input map)=(convolute)=>Feature detector=Feature Map
#places where high number rises in feature map are where ourf CNN can 
#find specific feature

In [5]:
#nb_filter=32,no.of rows,col in feature detector=3,input_shape(no. of channel,rows,col)
classifier.add(Convolution2D(32, 3, 3,input_shape= (64,64,3),activation='relu'))

C:\Users\virendra\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  


In [6]:
#pooling-taking max of 2,2 table of feature map(for reducing number of nodes)

In [7]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [8]:
#flattening-taking 2D pool table into single dim list

In [9]:
classifier.add(Flatten())

In [10]:
#ANN building

In [11]:
classifier.add(Dense(output_dim=128,activation='relu'))
classifier.add(Dense(output_dim=1,activation='sigmoid'))

C:\Users\virendra\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """Entry point for launching an IPython kernel.
C:\Users\virendra\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  


In [17]:
#compiling the CNN

In [18]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
#fitting the CNN to the images

In [20]:
from keras.preprocessing.image import ImageDataGenerator

In [21]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [22]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
training_set = train_datagen.flow_from_directory('train', target_size=(64, 64),batch_size=32,class_mode='binary')

Found 8000 images belonging to 2 classes.


In [25]:
test_set = test_datagen.flow_from_directory('test2',target_size=(64, 64),batch_size=32,class_mode='binary')

Found 2001 images belonging to 2 classes.


In [ ]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=2000,
        epochs=10,
        validation_data=test_set,
        nb_val_samples=2000)

C:\Users\virendra\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\virendra\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=2000, epochs=10, validation_data=<keras_pre..., validation_steps=2000)`
  


Epoch 1/10
2000/2000 [==============================] - 1010s 505ms/step - loss: 0.3581 - acc: 0.8399 - val_loss: 0.6468 - val_acc: 0.7356
Epoch 2/10
2000/2000 [==============================] - 1225s 613ms/step - loss: 0.2844 - acc: 0.8784 - val_loss: 0.6766 - val_acc: 0.7656
Epoch 3/10
2000/2000 [==============================] - 1099s 550ms/step - loss: 0.2303 - acc: 0.9056 - val_loss: 0.7581 - val_acc: 0.7446
Epoch 4/10
2000/2000 [==============================] - 849s 424ms/step - loss: 0.1877 - acc: 0.9247 - val_loss: 0.8010 - val_acc: 0.7696
Epoch 5/10
2000/2000 [==============================] - 1189s 594ms/step - loss: 0.1569 - acc: 0.9385 - val_loss: 0.9643 - val_acc: 0.7561
Epoch 6/10
2000/2000 [==============================] - 1011s 505ms/step - loss: 0.1287 - acc: 0.9507 - val_loss: 1.0645 - val_acc: 0.7516
Epoch 7/10
2000/2000 [==============================] - 862s 431ms/step - loss: 0.1129 - acc: 0.9572 - val_loss: 1.0476 - val_acc: 0.7586
Epoch 8/10
2000/2000 [=======

In [ ]:
# serialize model to JSON
classifier_json = classifier.to_json()
with open("classifier.json", "w") as json_file:
    json_file.write(classifier_json)
# serialize weights to HDF5
classifier.save_weights("classifier.h5")
print("Saved model to disk")

In [8]:
from keras.models import model_from_json
json_file = open('classifier.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("classifier.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
#single pridiction
import numpy as np
from keras.preprocessing import image

In [23]:
test_image=image.load_img("cd.jpg",target_size=(64,64))

In [24]:
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)

In [25]:
loaded_model.predict(test_image)

array([[1.]], dtype=float32)

In [26]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images
load_images_from_folder("p")

[array([[[154, 165, 169],
         [154, 165, 169],
         [153, 164, 168],
         ...,
         [147, 145, 144],
         [146, 144, 143],
         [146, 144, 143]],
 
        [[155, 166, 170],
         [154, 165, 169],
         [154, 165, 169],
         ...,
         [147, 145, 144],
         [145, 143, 142],
         [146, 144, 143]],
 
        [[155, 166, 170],
         [155, 166, 170],
         [155, 166, 170],
         ...,
         [148, 149, 147],
         [146, 147, 145],
         [147, 148, 146]],
 
        ...,
 
        [[ 11,  12,  16],
         [ 11,  12,  16],
         [ 12,  13,  17],
         ...,
         [185, 196, 200],
         [186, 197, 201],
         [185, 196, 200]],
 
        [[ 10,  11,  15],
         [ 12,  13,  17],
         [ 14,  15,  19],
         ...,
         [184, 195, 199],
         [183, 194, 198],
         [182, 193, 197]],
 
        [[ 10,  11,  15],
         [ 15,  16,  20],
         [ 17,  18,  22],
         ...,
         [182, 193, 197],
  

In [ ]:
c=0
